In [ ]:
# open chrome window using selenium without headless mode and navigate to https://telega.io
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import pandas as pd

# Set the path to the ChromeDriver you downloaded
chrome_driver_path = "/Users/yueliu/Downloads/chromedriver"

# Set up Chrome options (e.g., run headless for scraping without opening a browser)
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Optional: use for headless mode (no browser UI)

# Create a new Chrome session
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navigate to the website
driver.get('https://telega.io/catalog')

login_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//*[text()='Log in']"))
    )
login_element.click()

# input email and password
email_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//*[@id='form_sing_in']/div[1]/div[1]/div[1]/input"))
    )
email_element.send_keys("benliu20131124@gmail.com")

pw_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//*[@id='user_password']"))
    )
pw_element.send_keys("q6f5sljx")

login_button_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//*[@id='form_sing_in']/div[3]/div[2]"))
    )
login_button_element.click()
driver.get('https://telega.io/catalog')

# select topic

In [ ]:
topic = "Trading"

In [ ]:
all_topics_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//*[text()='All topics']"))
    )
all_topics_element.click()

# Wait for the "Cryptocurrencies" option to be visible and click it
crypto_element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f"//*[text()='{topic}']"))
)
crypto_element.click()

# get channel list

In [ ]:

data = []
seen_channels = set()

for i in range(10):
    if i > 0:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        try:
            all_topics_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//*[text()='More channels']"))
                )
            all_topics_element.click()
        except:
            break
    soup = BeautifulSoup(driver.page_source)
    all_channels = soup.find_all('div', class_='catalog_channels')[0].find_all(id=re.compile(r'^channels:\d+$'))
    for channel in all_channels:
        
        # Extract the channel name
        channel_name = channel.find('span', class_='font-18px bold').text.strip()
        if channel_name in seen_channels:
            continue
        print(channel_name)
        # Extract the number of followers
        followers = channel.find('div', class_='channel-users-count').find('span', class_='font-18px desck color-headline').text.strip()

        # Extract the price
        price = channel.find('div', class_='current_price text_price js-price').find('span', class_='amount en').text.strip()
        
        # telego channel link
        channel_link = 'https://telega.io'+channel.find('a', class_='about-avatar')['href']
        
        # extract view
        # view = channel.find('span', class_='font-18px desck color-headline js-view').text.strip()
        view = None
        male_ratio = channel.find('div', class_='progressbar').find('span')['style'].replace('width: ', '')
        reach_ratio = channel.find('span', class_='font-18px desck color-headline js-err').text
        # visit the link
        # driver.get(channel_link)
        
        # channel_page_soup = BeautifulSoup(driver.page_source)
        # tg_link = channel_page_soup.find('a', class_='about-link').text
        row = {'channel_name': channel_name, 'followers': followers, 'price': price, 'telega_link': channel_link,
               'view': view, 'male_ratio': male_ratio, 'reach_ratio': reach_ratio}
        data.append(row)
        seen_channels.add(channel_name)
        print(row)
        # go back to previous page
        # driver.back()

df = pd.DataFrame(data)
print(df.shape)
display(df.head())

# get channel details

In [ ]:
tglinks = []
langs = []
views = []
skip_up_to = 0

def extract_channel_info(soup):
    tg_link = channel_page_soup.find('a', class_='about-link').text
    language = channel_page_soup.find('div', class_='lang').find('div', class_='body-bold-x4').text
    view = channel_page_soup.find('div', class_="about-head-view tooltip-top tooltipstered").text
    return tg_link, language, view


for index, row in df.iterrows():
    print(index)
    html_file_path = f'channel_card_html/{row["telega_link"].replace("https://telega.io/channels/", "").replace("/card", "")}.html'
    if index < skip_up_to:
        with open(html_file_path, 'r') as f:
            html = f.read()
        channel_page_soup = BeautifulSoup(html)
    else:
        driver.get(row['telega_link'])
        channel_page_soup = BeautifulSoup(driver.page_source)
        with open(html_file_path, 'w') as f:
            f.write(driver.page_source)
    tg_link, language, view = extract_channel_info(channel_page_soup)
    tglinks.append(tg_link)
    langs.append(language)
    views.append(view)
    print(tg_link)


# write results

In [ ]:
df['tg_link'] = tglinks
df['language'] = langs
df['view'] = views
df.to_csv(f'{topic.lower().replace(" ", "_")}_channels.csv', index=False)